In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/AmExpert2019_data")
os.getcwd()

!pip install catboost
!pip install category-encoders

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install featuretools

In [0]:
# importing all libraries
from pandas import Series, DataFrame 
#import lightgbm and xgboost 
import lightgbm as lgb 
import xgboost as xgb 

import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from matplotlib import pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression
from scipy.special import logit

from catboost import CatBoostClassifier, Pool,cv

In [0]:
# loading all dataframes
campaign_data = pd.read_csv('campaign_data.csv')
coupon_item_mapping = pd.read_csv('coupon_item_mapping.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')
customer_transaction_data = pd.read_csv('customer_transaction_data.csv')
item_data = pd.read_csv('item_data.csv')
test_data = pd.read_csv('test_QyjYwdj.csv')
train_data = pd.read_csv('train.csv')

**Data field transformations**

In [0]:
customer_transaction_data['date'] = pd.to_datetime(customer_transaction_data['date'])
campaign_data['start_date'] = pd.to_datetime(campaign_data['start_date'], format='%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data['end_date'], format='%d/%m/%y')

**Combining all datasets** 

Steps to combine all datasets

1.   Left join-> train & cust_demo
2.   Left join-> train & campaign_data
3. Inner join-> couponitem & item data
4. aggregate over 3 on coupon_id
5.Inner join train and 3-1 on coupon_id
6.Inner join-> cust_trans_data & item data
7-1. aggregate on customer id
8.left join-> train & 5-1 on customer id

Note: please aggregate in separate dataframes and then join to train. This will avoid confusion and record mismatch

In [0]:
# creaing master data set
test_data['redemption_status'] = np.nan # putting nans to test data set 
master_data = pd.concat([train_data, test_data]) # combining train test (concat)
# master_data = pd.merge(master_data, customer_demographics, on='customer_id', how='left') # combining train and customer demo
master_data = pd.merge(master_data, campaign_data, on='campaign_id', how='left') # combining master and campaign data


In [0]:
# creating customer demographics features
customer_demographics_num = customer_demographics.copy()

customer_demographics_num['no_of_children'] = customer_demographics_num['no_of_children'].replace('3+', 3).astype(float)
customer_demographics_num['family_size'] = customer_demographics_num['family_size'].replace('5+', 3).astype(float)
customer_demographics_num['marital_status'] = pd.Series(customer_demographics_num['marital_status'].factorize()[0]).replace(-1, np.nan)
customer_demographics_num['age_range'] = pd.Series(customer_demographics_num['age_range'].factorize()[0]).replace(-1, np.nan)


# rented
rented_mean = customer_demographics_num.groupby("customer_id")['rented'].mean().to_dict()
master_data['rented_mean'] = master_data['customer_id'].map(rented_mean)
# income_bracket
income_bracket_sum = customer_demographics_num.groupby("customer_id")['income_bracket'].sum().to_dict()
master_data['income_bracket_sum'] = master_data['customer_id'].map(income_bracket_sum)
# age_range
age_range_mean = customer_demographics_num.groupby("customer_id")['age_range'].mean().to_dict()
master_data['age_range_mean'] = master_data['customer_id'].map(age_range_mean)
# family_size
family_size_mean = customer_demographics_num.groupby("customer_id")['family_size'].mean().to_dict()
master_data['family_size_mean'] = master_data['customer_id'].map(family_size_mean)
# no_of_children
no_of_children_mean = customer_demographics_num.groupby("customer_id")['no_of_children'].mean().to_dict()
master_data['no_of_children_mean'] = master_data['customer_id'].map(no_of_children_mean)
no_of_children_count = customer_demographics_num.groupby("customer_id")['no_of_children'].count().to_dict()
master_data['no_of_children_count'] = master_data['customer_id'].map(no_of_children_count)
# marital_status
marital_status_count = customer_demographics_num.groupby("customer_id")['marital_status'].count().to_dict()
master_data['marital_status_count'] = master_data['customer_id'].map(marital_status_count)

In [0]:
# creating count coupon id and campaign days feature 
df_temp = train_data.groupby('coupon_id').size().reset_index().rename(columns={0:'count_coupon_id'})
master_data = pd.merge(master_data, df_temp, on='coupon_id', how='left')
master_data['campaign_days'] = master_data['end_date']-master_data['start_date']
master_data['campaign_days'] = master_data['campaign_days'].dt.days.astype('int16')

# creating the redemption realted features from master_data
# campaign_type_redeem_index
# customer_id_redeem_index
# unique_coupon_id
# total_coupon_redeem

df1 = master_data[master_data.redemption_status == 1].campaign_type.value_counts().reset_index().rename(columns={'index':'campaign_type',\
                                                                                                                 'campaign_type':'campaign_type_redeem_index'})
df1['campaign_type_redeem_index'] = df1['campaign_type_redeem_index']/df1.campaign_type_redeem_index.sum()

df2 = master_data[master_data.redemption_status == 1].customer_id.value_counts().reset_index().rename(columns={'index':'customer_id',\
                                                                                                               'customer_id':'customer_id_redeem_index'})
df2['customer_id_redeem_index'] = df2['customer_id_redeem_index']/df2['customer_id_redeem_index'].sum()

df3 = master_data[~(master_data.redemption_status.isnull())].groupby(['customer_id']).agg({'coupon_id':'nunique', 'redemption_status':'sum'}).\
      reset_index().rename(columns={'coupon_id':'unique_coupon_id','redemption_status':'total_coupon_redeem'})
df3['redeem_per_coupon']= df3['total_coupon_redeem']/df3['unique_coupon_id']


# combining back to master data
master_data = pd.merge(master_data, df1, on='campaign_type', how='left')
master_data = pd.merge(master_data, df2, on='customer_id', how='left')
master_data = pd.merge(master_data, df3, on='customer_id', how='left')

In [9]:
# Combining Coupon and Item data and creating features on coupon level
# unique_items, unique_brands, unique_category, unique_brand_type
# Most occuring brands, category and brand type 
coupon_item_comb = pd.merge(coupon_item_mapping, item_data, on='item_id', how='inner')

coupon_data_grouped = coupon_item_comb.groupby(['coupon_id']).agg({'item_id':'nunique','brand':'nunique','category':'nunique', 'brand_type':'nunique'}).\
                      reset_index().rename(columns={'item_id':'unique_items','brand':'unique_brands','category':'unique_category', 'brand_type':'unique_brand_type'})


df = coupon_item_comb.groupby(["coupon_id"])["brand"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'brand':'mostoccuredbrand'})
coupon_data_grouped = pd.merge(coupon_data_grouped, df, on='coupon_id', how='left')

df = coupon_item_comb.groupby(["coupon_id"])["category"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'category':'mostoccuredcategory'})
coupon_data_grouped = pd.merge(coupon_data_grouped, df, on='coupon_id', how='left')

df = coupon_item_comb.groupby(["coupon_id"])["brand_type"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'brand_type':'mostoccuredbrand_type'})
coupon_data_grouped = pd.merge(coupon_data_grouped, df, on='coupon_id', how='left')

# creating binary category variables
temp1 = coupon_item_comb[['coupon_id','category']].drop_duplicates()
temp2 = pd.crosstab(index=temp1['coupon_id'], columns=[temp1['category']]).reset_index()
temp2 = pd.DataFrame(temp2)
coupon_data_grouped = pd.merge(coupon_data_grouped, temp2, on='coupon_id', how='left')

# creating brand type mean
temp3 = coupon_item_comb[['coupon_id', 'brand_type']]
temp3["typebrand"] = temp3['brand_type'].str.replace('Established','1')
temp3["typebrand"] = temp3['typebrand'].str.replace('Local','0')
temp3["typebrand"] = temp3["typebrand"].astype(int)
temp4 = temp3.groupby("coupon_id")["typebrand"].mean().reset_index().rename(columns={'typebrand':'typebrand_mean'})
coupon_data_grouped = pd.merge(coupon_data_grouped,temp4,on='coupon_id', how='left')


# creating the numerical brand, category features 
#coupon_item_mapping, item_data
coupon_item_comb['category_num'] = pd.Series(coupon_item_comb['category'].factorize()[0]).replace(-1, np.nan)

category = coupon_item_comb.groupby("coupon_id")['category_num'].mean().to_dict()
coupon_data_grouped['category_mean'] = coupon_data_grouped['coupon_id'].map(category)
category = coupon_item_comb.groupby("coupon_id")['category_num'].count().to_dict()
coupon_data_grouped['category_count'] = coupon_data_grouped['coupon_id'].map(category)
# category = coupon_item_comb.groupby("coupon_id")['category_num'].nunique().to_dict()
# coupon_data_grouped['category_nunique'] = coupon_data_grouped['coupon_id'].map(category)
category = coupon_item_comb.groupby("coupon_id")['category_num'].max().to_dict()
coupon_data_grouped['category_max'] = coupon_data_grouped['coupon_id'].map(category)
category = coupon_item_comb.groupby("coupon_id")['category_num'].min().to_dict()
coupon_data_grouped['category_min'] = coupon_data_grouped['coupon_id'].map(category)

brand_mean = coupon_item_comb.groupby("coupon_id")['brand'].mean().to_dict()
coupon_data_grouped['brand_mean'] = coupon_data_grouped['coupon_id'].map(brand_mean)
brand_mean = coupon_item_comb.groupby("coupon_id")['brand'].count().to_dict()
coupon_data_grouped['brand_count'] = coupon_data_grouped['coupon_id'].map(brand_mean)
brand_mean = coupon_item_comb.groupby("coupon_id")['brand'].min().to_dict()
coupon_data_grouped['brand_min'] = coupon_data_grouped['coupon_id'].map(brand_mean)
brand_mean = coupon_item_comb.groupby("coupon_id")['brand'].max().to_dict()
coupon_data_grouped['brand_max'] = coupon_data_grouped['coupon_id'].map(brand_mean)
# brand_mean = coupon_item_comb.groupby("coupon_id")['brand'].nunique().to_dict()
# coupon_data_grouped['brand_nunique'] = coupon_data_grouped['coupon_id'].map(brand_mean)

master_data = pd.merge(master_data, coupon_data_grouped, on='coupon_id', how='left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

### **Transaction level features**

In [0]:
# transaction customer id level features
trans_item_merge = customer_transaction_data.copy()
trans_item_merge = pd.merge(trans_item_merge, item_data, on='item_id',how='left')

# new columns creation 
trans_item_merge['coupon_discount'] = trans_item_merge['coupon_discount'].apply(lambda x: -x if x!=0 else x)
trans_item_merge['other_discount'] = trans_item_merge['other_discount'].apply(lambda x: -x if x!=0 else x)
trans_item_merge['total_discount'] = trans_item_merge['coupon_discount'] + trans_item_merge['other_discount']
trans_item_merge['original_price'] = trans_item_merge['selling_price']+trans_item_merge['other_discount']+trans_item_merge['coupon_discount']
trans_item_merge["MRP"] = (trans_item_merge["selling_price"]+trans_item_merge["other_discount"])/trans_item_merge["quantity"]
trans_item_merge["correctSellingPrice"] = trans_item_merge["selling_price"] - trans_item_merge["coupon_discount"]

trans_item_merge['coupon_dis_per'] = (trans_item_merge['coupon_discount']/trans_item_merge['original_price'])*100
trans_item_merge['other_dis_per'] = (trans_item_merge['other_discount']/trans_item_merge['original_price'])*100
trans_item_merge['total_dis_per'] = (trans_item_merge['total_discount']/trans_item_merge['original_price'])*100

trans_item_merge["MRP>category"] = trans_item_merge.groupby("category")["MRP"].transform('mean')
trans_item_merge["MRP>category"] = np.where(trans_item_merge["MRP"]>trans_item_merge["MRP>category"], 1, 0)

trans_item_merge["typebrand/customer"] = trans_item_merge.brand_type.str.replace('Established','1')
trans_item_merge["typebrand/customer"] = trans_item_merge["typebrand/customer"].str.replace('Local','0')
trans_item_merge["typebrand/customer"] = trans_item_merge["typebrand/customer"].astype(int)

trans_item_merge['day'] = trans_item_merge.date.dt.day

# grouping at customer id level
# no_transaction,total_quantity,  total_selling_price, total_other_discount, total_coupon_discount, total_total_discount, unique_items_cus, unique_brand_cus
# unique_category_cus
df4 = trans_item_merge.groupby(['customer_id']).agg({'date':'count', 'quantity':'sum','selling_price':'sum','other_discount':'sum','coupon_discount':'sum',\
'total_discount':'sum', 'item_id':'nunique','brand':'nunique', 'category':'nunique'})\
.reset_index().rename(columns={'date':'no_transaction', 'quantity':'total_quantity','selling_price':'total_selling_price','other_discount':'total_other_discount',\
'coupon_discount':'total_coupon_discount','total_discount':'total_total_discount', 'item_id':'unique_items_cus','brand':'unique_brand_cus','category':'unique_category_cus'})

df41 = trans_item_merge[trans_item_merge.coupon_discount!=0].groupby(['customer_id']).agg({'coupon_dis_per':'mean'})\
.reset_index().rename(columns={'coupon_dis_per':'coupon_dis_per_avg'})

df42 = trans_item_merge[trans_item_merge.other_discount!=0].groupby(['customer_id']).agg({'other_dis_per':'mean'})\
.reset_index().rename(columns={'other_dis_per':'other_dis_per_avg'})

df43 = trans_item_merge[trans_item_merge.total_discount!=0].groupby(['customer_id']).agg({'total_dis_per':'mean'})\
.reset_index().rename(columns={'total_dis_per':'total_dis_per_avg'})


# quantity_mean, coupon_discount_mean, other_discount_mean, date_day_mean
quantity_mean = trans_item_merge.groupby("customer_id")['quantity'].mean().reset_index().rename(columns={'quantity':'quantity_mean'})
coupon_discount_mean = trans_item_merge.groupby("customer_id")['coupon_discount'].mean().reset_index().rename(columns={'coupon_discount':'coupon_discount_mean'})
other_discount_mean = trans_item_merge.groupby("customer_id")['other_discount'].mean().reset_index().rename(columns={'other_discount':'other_discount_mean'})
date_day_mean = trans_item_merge.groupby("customer_id")['day'].mean().reset_index().rename(columns={'day':'day_mean'})

# selling_price_mean, selling_price_mean, selling_price_min, selling_price_max,selling_price_nunique
selling_price_mean = trans_item_merge.groupby("customer_id")['selling_price'].mean().reset_index().rename(columns={'selling_price':'selling_price_mean'})
# selling_price_sum = customer_transaction_data.groupby("customer_id")['selling_price'].sum().reset_index().rename(columns={'selling_price':'selling_price_sum'})
selling_price_min = trans_item_merge.groupby("customer_id")['selling_price'].min().reset_index().rename(columns={'selling_price':'selling_price_min'})
selling_price_max = trans_item_merge.groupby("customer_id")['selling_price'].max().reset_index().rename(columns={'selling_price':'selling_price_max'})
selling_price_nunique = trans_item_merge.groupby("customer_id")['selling_price'].nunique().reset_index().rename(columns={'selling_price':'selling_price_unique'})

# MRP_mean, sum, min, max, nunique
MRP_mean = trans_item_merge.groupby("customer_id")['MRP'].mean().reset_index().rename(columns={'MRP':'MRP_mean'})
MRP_sum = trans_item_merge.groupby("customer_id")['MRP'].sum().reset_index().rename(columns={'MRP':'MRP_sum'})
MRP_min = trans_item_merge.groupby("customer_id")['MRP'].min().reset_index().rename(columns={'MRP':'MRP_min'})
MRP_max = trans_item_merge.groupby("customer_id")['MRP'].max().reset_index().rename(columns={'MRP':'MRP_max'})
MRP_nunique = trans_item_merge.groupby("customer_id")['MRP'].nunique().reset_index().rename(columns={'MRP':'MRP_unique'})

# correctSellingPrice Mean, max, min, nunique
correctSellingPrice_mean = trans_item_merge.groupby("customer_id")['correctSellingPrice'].mean().reset_index().rename(columns={'correctSellingPrice':'correctSellingPrice_mean'})
correctSellingPrice_sum = trans_item_merge.groupby("customer_id")['correctSellingPrice'].sum().reset_index().rename(columns={'correctSellingPrice':'correctSellingPrice_sum'})
correctSellingPrice_min = trans_item_merge.groupby("customer_id")['correctSellingPrice'].min().reset_index().rename(columns={'correctSellingPrice':'correctSellingPrice_min'})
correctSellingPrice_max = trans_item_merge.groupby("customer_id")['correctSellingPrice'].max().reset_index().rename(columns={'correctSellingPrice':'correctSellingPrice_max'})
correctSellingPrice_nunique = trans_item_merge.groupby("customer_id")['correctSellingPrice'].nunique().reset_index().rename(columns={'correctSellingPrice':'correctSellingPrice_unique'})

# mean when average MRP is greater than Category mean
MRP_cat_mean = trans_item_merge.groupby("customer_id")['MRP>category'].mean().reset_index().rename(columns={'MRP>category':'MRP_cat_mean'})

typebrand_per_customer_mean = trans_item_merge.groupby("customer_id")['typebrand/customer'].mean().reset_index().rename(columns={'typebrand/customer':'typebrand_per_customer_mean'})


# combining back to master data
df4 = pd.merge(df4,df41,on='customer_id',how='left')
df4 = pd.merge(df4,df42,on='customer_id',how='left')
df4 = pd.merge(df4,df43,on='customer_id',how='left')
master_data = pd.merge(master_data, df4, on='customer_id',how='left')

master_data = pd.merge(master_data, quantity_mean, on='customer_id', how='left')
master_data = pd.merge(master_data, coupon_discount_mean, on='customer_id', how='left')
master_data = pd.merge(master_data, other_discount_mean, on='customer_id', how='left')
master_data = pd.merge(master_data, date_day_mean, on='customer_id', how='left')

master_data = pd.merge(master_data, selling_price_mean, on='customer_id', how='left')
# master_data = pd.merge(master_data, selling_price_sum, on='customer_id', how='left')
master_data = pd.merge(master_data, selling_price_min, on='customer_id', how='left')
master_data = pd.merge(master_data, selling_price_max, on='customer_id', how='left')
master_data = pd.merge(master_data, selling_price_nunique, on='customer_id', how='left')

master_data = pd.merge(master_data, MRP_mean, on='customer_id', how='left')
master_data = pd.merge(master_data, MRP_sum, on='customer_id', how='left')
master_data = pd.merge(master_data, MRP_min, on='customer_id', how='left')
master_data = pd.merge(master_data, MRP_max, on='customer_id', how='left')
master_data = pd.merge(master_data, MRP_nunique, on='customer_id', how='left')

master_data = pd.merge(master_data, correctSellingPrice_mean, on='customer_id', how='left')
master_data = pd.merge(master_data, correctSellingPrice_sum, on='customer_id', how='left')
master_data = pd.merge(master_data, correctSellingPrice_min, on='customer_id', how='left')
master_data = pd.merge(master_data, correctSellingPrice_max, on='customer_id', how='left')
master_data = pd.merge(master_data, correctSellingPrice_nunique, on='customer_id', how='left')

master_data = pd.merge(master_data, MRP_cat_mean, on='customer_id', how='left')
master_data = pd.merge(master_data, typebrand_per_customer_mean, on='customer_id', how='left')


#Most purchased items, brand, category
df = trans_item_merge.groupby(["customer_id"])["item_id"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'item_id':'mostpurchaseditem'})
master_data = pd.merge(master_data, df, on='customer_id', how='left')
 

df = trans_item_merge.groupby(["customer_id"])["brand"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'brand':'mostpurchasedbrand'})
master_data = pd.merge(master_data, df, on='customer_id', how='left')
 

df = trans_item_merge.groupby(["customer_id"])["category"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'category':'mostpurchasedcategory'})
master_data = pd.merge(master_data, df, on='customer_id', how='left')

  
df = trans_item_merge[trans_item_merge['total_discount']!=0].groupby("customer_id")["date"].count().reset_index().rename(columns={'date':'total_discounted_trans'})
master_data = pd.merge(master_data, df, on='customer_id', how='left')

This article develops a model of the consumer's decision to redeem a coupon on a purchase occasion. The redemption decision is a function of four types of variables - coupon characteristics, characteristics of the purchase, brand loyalty, and concurrent promotional conditions. 

**Coupon level features in Transaction table**

In [0]:
df = coupon_item_comb.groupby(["coupon_id"])["brand"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'brand':'mostoccuredbrand'})
coupon_data_grouped = pd.merge(coupon_data_grouped, df, on='coupon_id', how='left')

df = coupon_item_comb.groupby(["coupon_id"])["category"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'category':'mostoccuredcategory'})
coupon_data_grouped = pd.merge(coupon_data_grouped, df, on='coupon_id', how='left')

df = coupon_item_comb.groupby(["coupon_id"])["brand_type"].agg(lambda x:x.value_counts().index[0]).reset_index().rename(columns={'brand_type':'mostoccuredbrand_type'})
coupon_data_grouped = pd.merge(coupon_data_grouped, df, on='coupon_id', how='left')


In [0]:
print(test_data.shape)
df_temp = test_data[['customer_id','coupon_id']].drop_duplicates()
print(df_temp.shape)

(50226, 5)
(49649, 2)


In [0]:
trans_item_merge.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,total_discount,original_price,MRP,correctSellingPrice,coupon_dis_per,other_dis_per,total_dis_per,MRP>category,typebrand/customer,day
0,2012-01-02,1501,26830,1,35.26,10.69,0.0,56,Local,Natural Products,10.69,45.95,45.95,35.26,0.0,23.264418,23.264418,0,0,2
1,2012-01-02,1501,54253,1,53.43,13.89,0.0,56,Local,Natural Products,13.89,67.32,67.32,53.43,0.0,20.632799,20.632799,0,0,2
2,2012-01-02,1501,31962,1,106.50,14.25,0.0,524,Established,Grocery,14.25,120.75,120.75,106.50,0.0,11.801242,11.801242,1,1,2
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery,0.00,67.32,67.32,67.32,0.0,0.000000,0.000000,0,1,2
4,2012-01-02,1501,48199,1,71.24,28.14,0.0,524,Established,Grocery,28.14,99.38,99.38,71.24,0.0,28.315556,28.315556,1,1,2


In [0]:
coupon_item_mapping.head()
# coupon_data_grouped.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


# **Adding features using  Feature tools **

In [0]:
import featuretools as ft

In [0]:

# loading all dataframes
train = pd.read_csv("train.csv")
test = pd.read_csv("test_QyjYwdj.csv")
campaign = pd.read_csv("campaign_data.csv")
coupon = pd.read_csv("coupon_item_mapping.csv")
demographics = pd.read_csv("customer_demographics.csv")
transaction = pd.read_csv("customer_transaction_data.csv")
item = pd.read_csv("item_data.csv")

In [58]:
demographics['no_of_children'] = demographics['no_of_children'].replace('3+', 3).astype(float)
demographics['family_size'] = demographics['family_size'].replace('5+', 3).astype(float)
demographics['marital_status'] = pd.Series(demographics['marital_status'].factorize()[0]).replace(-1, np.nan)
demographics['age_range'] = pd.Series(demographics['age_range'].factorize()[0]).replace(-1, np.nan)
demographics.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,0,0.0,0,2.0,NaN,4
1,6,1,0.0,0,2.0,NaN,5
2,7,2,NaN,0,3.0,1.0,3
3,8,2,NaN,0,4.0,2.0,6
4,10,1,1.0,0,1.0,NaN,5


In [13]:
demographics.reset_index(inplace=True)
es = ft.EntitySet(id = 'demographics')
es.entity_from_dataframe(entity_id = 'demographics', dataframe = demographics, index = 'index')
es.normalize_entity(base_entity_id='demographics', new_entity_id='customer', index = 'customer_id')

feature_matrix_demo, feature_names_1 = ft.dfs(entityset=es, 
target_entity = 'customer', 
max_depth = 3, 
verbose = 1, 
n_jobs = 1)
feature_matrix_demo.reset_index(inplace=True)


Built 37 features
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks


In [36]:
feature_matrix_demo.head()

,customer_id,SUM(demographics.age_range),SUM(demographics.marital_status),SUM(demographics.rented),SUM(demographics.family_size),SUM(demographics.no_of_children),SUM(demographics.income_bracket),STD(demographics.age_range),STD(demographics.marital_status),STD(demographics.rented),STD(demographics.family_size),STD(demographics.no_of_children),STD(demographics.income_bracket),MAX(demographics.age_range),MAX(demographics.marital_status),MAX(demographics.rented),MAX(demographics.family_size),MAX(demographics.no_of_children),MAX(demographics.income_bracket),SKEW(demographics.age_range),SKEW(demographics.marital_status),SKEW(demographics.rented),SKEW(demographics.family_size),SKEW(demographics.no_of_children),SKEW(demographics.income_bracket),MIN(demographics.age_range),MIN(demographics.marital_status),MIN(demographics.rented),MIN(demographics.family_size),MIN(demographics.no_of_children),MIN(demographics.income_bracket),MEAN(demographics.age_range),MEAN(demographics.marital_status),MEAN(demographics.rented),MEAN(demographics.family_size),MEAN(demographics.no_of_children),MEAN(demographics.income_bracket),COUNT(demographics)
0,1,0,0.0,0,2.0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0,2.0,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0,2.0,NaN,4,0,0.0,0,2.0,NaN,4,1
1,6,1,0.0,0,2.0,0.0,5,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0,2.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0,2.0,NaN,5,1,0.0,0,2.0,NaN,5,1
2,7,2,0.0,0,3.0,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0,3.0,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0,3.0,1.0,3,2,NaN,0,3.0,1.0,3,1
3,8,2,0.0,0,4.0,2.0,6,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0,4.0,2.0,6,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0,4.0,2.0,6,2,NaN,0,4.0,2.0,6,1
4,10,1,1.0,0,1.0,0.0,5,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,1.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,0,1.0,NaN,5,1,1.0,0,1.0,NaN,5,1


In [59]:
test_data['redemption_status'] = np.nan # putting nans to test data set
train_test_combine = pd.concat([train, test]) # combining train test (concat)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [0]:
# #Campaign Data
# campaign['start_date']=pd.to_datetime(campaign['start_date'],format='%d/%m/%y')
# campaign['end_date']=pd.to_datetime(campaign['end_date'],format='%d/%m/%y')
# campaign["duration"] = campaign["start_date"]-campaign["end_date"]
# campaign["duration"] = campaign["duration"].apply(lambda x:x.days) 
# #Merging with campaign data
# train = pd.merge(train,campaign,on="campaign_id")

In [15]:
# for i in ['id', 'campaign_id', 'coupon_id', 'customer_id']:
#     train[i] = train[i].astype(str)
# train.dtypes

id                           object
campaign_id                  object
coupon_id                    object
customer_id                  object
redemption_status             int64
campaign_type                object
start_date           datetime64[ns]
end_date             datetime64[ns]
duration                      int64
dtype: object

In [9]:
#Item_ID cleaning
item["item_id"] = item["item_id"].astype(str)
item["brand"] = item["brand"].astype(str)
item["category"] = item["category"].astype(str)
item["brand_type"] = item.brand_type.str.replace('Established','1')
item["brand_type"] = item["brand_type"].str.replace('Local','0')
item["brand_type"] = item["brand_type"].astype(int)

#Item_ID merging
coupon = coupon.astype(str)
coupon = pd.merge(coupon,item,on="item_id")

coupon.dtypes, coupon.head()


(coupon_id     object
 item_id       object
 brand         object
 brand_type     int64
 category      object
 dtype: object,   coupon_id item_id brand  brand_type category
 0       105      37    56           0  Grocery
 1         6      37    56           0  Grocery
 2        22      37    56           0  Grocery
 3        31      37    56           0  Grocery
 4       107      75    56           0  Grocery)

In [10]:
coupon.reset_index(inplace=True)
es = ft.EntitySet(id = 'coupons')
es.entity_from_dataframe(entity_id = 'coupons', dataframe = coupon, index = 'index')
es.normalize_entity(base_entity_id='coupons', new_entity_id='cpon', index = 'coupon_id')

feature_matrix_1, feature_names_1 = ft.dfs(entityset=es, 
target_entity = 'cpon', 
max_depth = 3, 
verbose = 1, 
n_jobs = 1)
feature_matrix_1.reset_index(inplace=True)

Built 13 features
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [11]:
feature_matrix_1.head()

,coupon_id,SUM(coupons.brand_type),STD(coupons.brand_type),MAX(coupons.brand_type),SKEW(coupons.brand_type),MIN(coupons.brand_type),MEAN(coupons.brand_type),COUNT(coupons),NUM_UNIQUE(coupons.item_id),NUM_UNIQUE(coupons.brand),NUM_UNIQUE(coupons.category),MODE(coupons.item_id),MODE(coupons.brand),MODE(coupons.category)
0,1,39,0.0,1,0.0,1,1.0,39,39,3,2,17091,1475,Natural Products
1,10,11,0.0,1,0.0,1,1.0,11,11,3,2,18501,971,Grocery
2,100,0,0.0,0,0.0,0,0.0,5,5,1,1,15728,56,Grocery
3,1000,20,0.0,1,0.0,1,1.0,20,20,1,1,11212,1470,Pharmaceutical
4,1001,31,0.0,1,0.0,1,1.0,31,31,1,1,17091,1475,Natural Products


In [12]:
feature_matrix_1.shape

(1116, 14)

In [0]:
# Y = train["redemption_status"]
# #train.drop("redemption_status",inplace=True,axis=1)
# for i in ['id', 'campaign_id', 'coupon_id', 'customer_id']:
#     train[i] = train[i].astype(str)
# train = pd.merge(train,feature_matrix_1,on='coupon_id',how='left')

In [22]:
# train.drop("MODE(coupons.item_id)",axis=1,inplace=True)
# #train = train.head(200)
# es = ft.EntitySet(id = 'trainingdata')
# es.entity_from_dataframe(entity_id = 'trainingdata', dataframe = train, index = 'id')
# es.normalize_entity(base_entity_id='trainingdata', new_entity_id='campaign', index = 'campaign_id')
# es.normalize_entity(base_entity_id='trainingdata', new_entity_id='customer', index = 'customer_id')
# es.normalize_entity(base_entity_id='trainingdata', new_entity_id='coupon', index = 'coupon_id')

# feature_matrix_train, feature_names_train = ft.dfs(entityset=es, 
# target_entity = 'trainingdata', 
# max_depth = 3, 
# verbose = 1, 
# n_jobs = 1,ignore_variables={'trainingdata': ['redemption_status']})

# feature_matrix_train.reset_index(inplace=True)


Built 304 features
Elapsed: 03:59 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [0]:
# # feature_matrix_train.to_csv('feature_matrix_train.csv', index=False)
# feature_matrix_train = pd.read_csv('feature_matrix_train.csv')
# print(feature_matrix_train.shape)
# feature_matrix_train.head() 

In [0]:
feature_matrix_train.dtypes

Creating tools for traini and test

In [0]:
#Campaign Data
campaign['start_date']=pd.to_datetime(campaign['start_date'],format='%d/%m/%y')
campaign['end_date']=pd.to_datetime(campaign['end_date'],format='%d/%m/%y')
campaign["duration"] = campaign["start_date"]-campaign["end_date"]
campaign["duration"] = campaign["duration"].apply(lambda x:x.days) 
#Merging with campaign data
train_test_combine = pd.merge(train_test_combine,campaign,on="campaign_id")

In [65]:
demographics.customer_id.dtype, train_test_combine.customer_id.dtype

(dtype('int64'), dtype('int64'))

In [41]:
feature_matrix_1.coupon_id.dtype, train_test_combine.coupon_id.dtype

(dtype('O'), dtype('int64'))

In [0]:
demographics.customer_id = feature_matrix_demo.customer_id.astype(str)

In [0]:
# train_test_combine
Y = train_test_combine["redemption_status"]
#train.drop("redemption_status",inplace=True,axis=1)
for i in ['id', 'campaign_id', 'coupon_id', 'customer_id']:
    train_test_combine[i] = train_test_combine[i].astype(str)
    
    
train_test_combine = pd.merge(train_test_combine,feature_matrix_1,on='coupon_id',how='left')
train_test_combine = pd.merge(train_test_combine,demographics,on='customer_id',how='left')


In [68]:
for i in ['id', 'campaign_id', 'coupon_id', 'customer_id']:
    train_test_combine[i] = train_test_combine[i].astype(str)
train_test_combine.dtypes

campaign_id                             object
coupon_id                               object
customer_id                             object
id                                      object
redemption_status                      float64
campaign_type                           object
start_date                      datetime64[ns]
end_date                        datetime64[ns]
duration                                 int64
SUM(coupons.brand_type)                  int64
STD(coupons.brand_type)                float64
MAX(coupons.brand_type)                  int64
SKEW(coupons.brand_type)               float64
MIN(coupons.brand_type)                  int64
MEAN(coupons.brand_type)               float64
COUNT(coupons)                           int64
NUM_UNIQUE(coupons.item_id)              int64
NUM_UNIQUE(coupons.brand)                int64
NUM_UNIQUE(coupons.category)             int64
MODE(coupons.item_id)                   object
MODE(coupons.brand)                     object
MODE(coupons.

In [69]:
# train_test_combine.drop("MODE(coupons.item_id)",axis=1,inplace=True)
#train = train.head(200)
es = ft.EntitySet(id = 'trainingdata')
es.entity_from_dataframe(entity_id = 'trainingdata', dataframe = train_test_combine, index = 'id')
es.normalize_entity(base_entity_id='trainingdata', new_entity_id='campaign', index = 'campaign_id')
es.normalize_entity(base_entity_id='trainingdata', new_entity_id='customer', index = 'customer_id')
es.normalize_entity(base_entity_id='trainingdata', new_entity_id='coupon', index = 'coupon_id')

feature_matrix_train, feature_names_train = ft.dfs(entityset=es, 
target_entity = 'trainingdata', 
max_depth = 3, 
verbose = 1, 
n_jobs = 1,ignore_variables={'trainingdata': ['redemption_status']})

feature_matrix_train.reset_index(inplace=True)


Built 425 features
Elapsed: 06:29 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [13]:
# feature_matrix_train.to_csv('feature_matrix_train_test_combine_demo_normal.csv', index=False)
feature_matrix_train = pd.read_csv('feature_matrix_train_test_combine_demo_normal.csv')
print(feature_matrix_train.shape)
feature_matrix_train.head()

(128595, 426)


,id,campaign_id,coupon_id,customer_id,campaign_type,duration,SUM(coupons.brand_type),STD(coupons.brand_type),MAX(coupons.brand_type),SKEW(coupons.brand_type),MIN(coupons.brand_type),MEAN(coupons.brand_type),COUNT(coupons),NUM_UNIQUE(coupons.item_id),NUM_UNIQUE(coupons.brand),NUM_UNIQUE(coupons.category),MODE(coupons.item_id),MODE(coupons.brand),MODE(coupons.category),age_range,marital_status,rented,family_size,no_of_children,income_bracket,DAY(start_date),DAY(end_date),YEAR(start_date),YEAR(end_date),MONTH(start_date),MONTH(end_date),WEEKDAY(start_date),WEEKDAY(end_date),campaign.SUM(trainingdata.duration),campaign.SUM(trainingdata.SUM(coupons.brand_type)),campaign.SUM(trainingdata.STD(coupons.brand_type)),campaign.SUM(trainingdata.MAX(coupons.brand_type)),campaign.SUM(trainingdata.SKEW(coupons.brand_type)),campaign.SUM(trainingdata.MIN(coupons.brand_type)),campaign.SUM(trainingdata.MEAN(coupons.brand_type)),...,campaign.MODE(trainingdata.DAY(start_date)),campaign.MODE(trainingdata.DAY(end_date)),campaign.MODE(trainingdata.YEAR(start_date)),campaign.MODE(trainingdata.YEAR(end_date)),campaign.MODE(trainingdata.MONTH(start_date)),campaign.MODE(trainingdata.MONTH(end_date)),campaign.MODE(trainingdata.WEEKDAY(start_date)),campaign.MODE(trainingdata.WEEKDAY(end_date)),customer.NUM_UNIQUE(trainingdata.DAY(start_date)),customer.NUM_UNIQUE(trainingdata.DAY(end_date)),customer.NUM_UNIQUE(trainingdata.YEAR(start_date)),customer.NUM_UNIQUE(trainingdata.YEAR(end_date)),customer.NUM_UNIQUE(trainingdata.MONTH(start_date)),customer.NUM_UNIQUE(trainingdata.MONTH(end_date)),customer.NUM_UNIQUE(trainingdata.WEEKDAY(start_date)),customer.NUM_UNIQUE(trainingdata.WEEKDAY(end_date)),customer.MODE(trainingdata.DAY(start_date)),customer.MODE(trainingdata.DAY(end_date)),customer.MODE(trainingdata.YEAR(start_date)),customer.MODE(trainingdata.YEAR(end_date)),customer.MODE(trainingdata.MONTH(start_date)),customer.MODE(trainingdata.MONTH(end_date)),customer.MODE(trainingdata.WEEKDAY(start_date)),customer.MODE(trainingdata.WEEKDAY(end_date)),coupon.NUM_UNIQUE(trainingdata.DAY(start_date)),coupon.NUM_UNIQUE(trainingdata.DAY(end_date)),coupon.NUM_UNIQUE(trainingdata.YEAR(start_date)),coupon.NUM_UNIQUE(trainingdata.YEAR(end_date)),coupon.NUM_UNIQUE(trainingdata.MONTH(start_date)),coupon.NUM_UNIQUE(trainingdata.MONTH(end_date)),coupon.NUM_UNIQUE(trainingdata.WEEKDAY(start_date)),coupon.NUM_UNIQUE(trainingdata.WEEKDAY(end_date)),coupon.MODE(trainingdata.DAY(start_date)),coupon.MODE(trainingdata.DAY(end_date)),coupon.MODE(trainingdata.YEAR(start_date)),coupon.MODE(trainingdata.YEAR(end_date)),coupon.MODE(trainingdata.MONTH(start_date)),coupon.MODE(trainingdata.MONTH(end_date)),coupon.MODE(trainingdata.WEEKDAY(start_date)),coupon.MODE(trainingdata.WEEKDAY(end_date))
0,1,13,27,1053,X,-47,125,0.000000,1,0.000000,1,1.000000,125,125,2,1,10535,1105,Grocery,1.0,NaN,0.0,1.0,NaN,5.0,19,5,2013,2013,5,7,6,4,-1062482,3164965,293.594784,16197,-1228.607336,15489,16030.829404,...,19,5,2013,2013,5,7,6,4,2,2,1,1,2,2,2,1,19,5,2013,2013,5,7,6,4,1,1,1,1,1,1,1,1,19,5,2013,2013,5,7,6,4
1,10,17,498,811,Y,-32,18,0.000000,1,0.000000,1,1.000000,18,18,1,1,14259,209,Grocery,NaN,NaN,NaN,NaN,NaN,NaN,29,30,2013,2013,7,8,0,4,-122816,135744,0.000000,3636,0.000000,3636,3636.000000,...,29,30,2013,2013,7,8,0,4,5,5,1,1,5,4,3,1,10,4,2013,2013,8,10,0,4,1,1,1,1,1,1,1,1,29,30,2013,2013,7,8,0,4
2,100,18,234,433,X,-55,0,0.000000,0,0.000000,0,0.000000,5,5,1,1,18243,56,Grocery,NaN,NaN,NaN,NaN,NaN,NaN,10,4,2013,2013,8,10,5,4,-1290520,3273819,279.132799,16197,-1176.658264,15465,16018.197525,...,10,4,2013,2013,8,10,5,4,6,6,2,1,7,6,4,2,16,4,2013,2013,9,10,5,4,1,1,1,1,1,1,1,1,10,4,2013,2013,8,10,5,4
3,1000,18,33,154,X,-55,3897,0.316762,1,-2.443902,0,0.886891,4394,4394,1185,7,10010,56,Meat,1.0,0.0,0.0,3.0,3.0,6.0,10,4,2013,2013,8,10,5,4,-1290520,3273819,279.132799,16197,-1176.658264,15465,16018.197525,...,10,4,2013,2013,8,10,5,4,5,4,1,1,4,5,3,1,10,5,2013,2013,8,10,5,4,1,1,1,1,1,1,1,1,10,4,2013,2013,8,10,5,4
4,10

In [0]:
feature_matrix_train.isnull().sum()

In [0]:
# trans_item_merge.reset_index(inplace=True)
# trans_item_merge.head()

,index,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,total_discount,original_price,MRP,correctSellingPrice,coupon_dis_per,other_dis_per,total_dis_per,MRP>category,typebrand/customer,day
0,0,2012-01-02,1501,26830,1,35.26,10.69,0.0,56,Local,Natural Products,10.69,45.95,45.95,35.26,0.0,23.264418,23.264418,0,0,2
1,1,2012-01-02,1501,54253,1,53.43,13.89,0.0,56,Local,Natural Products,13.89,67.32,67.32,53.43,0.0,20.632799,20.632799,0,0,2
2,2,2012-01-02,1501,31962,1,106.50,14.25,0.0,524,Established,Grocery,14.25,120.75,120.75,106.50,0.0,11.801242,11.801242,1,1,2
3,3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery,0.00,67.32,67.32,67.32,0.0,0.000000,0.000000,0,1,2
4,4,2012-01-02,1501,48199,1,71.24,28.14,0.0,524,Established,Grocery,28.14,99.38,99.38,71.24,0.0,28.315556,28.315556,1,1,2


In [54]:
# es = ft.EntitySet(id = 'trans_item_merge')
# es.entity_from_dataframe(entity_id = 'trans_item_merge', dataframe = trans_item_merge, index = 'index')
# es.normalize_entity(base_entity_id='trans_item_merge', new_entity_id='customer', index = 'customer_id')
# es.normalize_entity(base_entity_id='trans_item_merge', new_entity_id='item', index = 'item_id',additional_variables = ["brand","brand_type","category"])

# feature_matrix_customer, feature_names_customer = ft.dfs(entityset=es, 
# target_entity = 'customer', 
# max_depth = 3, 
# verbose = 1, 
# n_jobs = 1)

Built 623 features

Elapsed: 00:00 | Remaining: ? | Progress:   0%|          | Calculated: 0/11 chunks
Elapsed: 02:47 | Remaining: 27:55 | Progress:   9%|▉         | Calculated: 1/11 chunks
Elapsed: 05:31 | Remaining: 24:58 | Progress:  18%|█▊        | Calculated: 2/11 chunks
Elapsed: 08:23 | Remaining: 22:26 | Progress:  27%|██▋       | Calculated: 3/11 chunks
Elapsed: 11:20 | Remaining: 19:55 | Progress:  36%|███▋      | Calculated: 4/11 chunks
Elapsed: 14:12 | Remaining: 17:07 | Progress:  45%|████▌     | Calculated: 5/11 chunks
Elapsed: 17:01 | Remaining: 14:12 | Progress:  55%|█████▍    | Calculated: 6/11 chunks
Elapsed: 19:52 | Remaining: 11:21 | Progress:  64%|██████▎   | Calculated: 7/11 chunks
Elapsed: 22:49 | Remaining: 08:37 | Progress:  73%|███████▎  | Calculated: 8/11 chunks
Elapsed: 25:38 | Remaining: 05:42 | Progress:  82%|████████▏ | Calculated: 9/11 chunks
Elapsed: 28:45 | Remaining: 02:56 | Progress:  91%|█████████ | Calculated: 10/11 chunks
Elapsed: 28:53 | Remaining

In [60]:
# feature_matrix_customer.reset_index(inplace=True)
# feature_matrix_customer.head()

,customer_id,SUM(trans_item_merge.quantity),SUM(trans_item_merge.selling_price),SUM(trans_item_merge.other_discount),SUM(trans_item_merge.coupon_discount),SUM(trans_item_merge.total_discount),SUM(trans_item_merge.original_price),SUM(trans_item_merge.MRP),SUM(trans_item_merge.correctSellingPrice),SUM(trans_item_merge.coupon_dis_per),SUM(trans_item_merge.other_dis_per),SUM(trans_item_merge.total_dis_per),SUM(trans_item_merge.MRP>category),SUM(trans_item_merge.typebrand/customer),SUM(trans_item_merge.day),STD(trans_item_merge.quantity),STD(trans_item_merge.selling_price),STD(trans_item_merge.other_discount),STD(trans_item_merge.coupon_discount),STD(trans_item_merge.total_discount),STD(trans_item_merge.original_price),STD(trans_item_merge.MRP),STD(trans_item_merge.correctSellingPrice),STD(trans_item_merge.coupon_dis_per),STD(trans_item_merge.other_dis_per),STD(trans_item_merge.total_dis_per),STD(trans_item_merge.MRP>category),STD(trans_item_merge.typebrand/customer),STD(trans_item_merge.day),MAX(trans_item_merge.quantity),MAX(trans_item_merge.selling_price),MAX(trans_item_merge.other_discount),MAX(trans_item_merge.coupon_discount),MAX(trans_item_merge.total_discount),MAX(trans_item_merge.original_price),MAX(trans_item_merge.MRP),MAX(trans_item_merge.correctSellingPrice),MAX(trans_item_merge.coupon_dis_per),MAX(trans_item_merge.other_dis_per),MAX(trans_item_merge.total_dis_per),...,MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.day)),MEAN(trans_item_merge.item.MIN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MIN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MIN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MIN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MIN(trans_item_merge.day)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.day)),MEAN(trans_item_merge.item.COUNT(trans_item_merge)),MEAN(trans_item_merge.item.NUM_UNIQUE(trans_item_merge.customer_id)),NUM_UNIQUE(trans_item_merge.item.MODE(trans_item_merge.customer_id)),MODE(trans_item_merge.item.MODE(trans_item_merge.customer_id))
0,1,1227,98513.93,17030.40,2116.83,19147.23,1176

In [0]:
# feature_matrix_customer.to_csv('feature_matrix_customer.csv', index=False)
feature_matrix_customer = pd.read_csv('feature_matrix_customer.csv')

In [0]:
# feature_matrix_1
# feature_matrix_train
# feature_matrix_customer



In [21]:
# feature_matrix_1.head()
# feature_matrix_1.shape
feature_matrix_customer.head()

,customer_id,SUM(trans_item_merge.quantity),SUM(trans_item_merge.selling_price),SUM(trans_item_merge.other_discount),SUM(trans_item_merge.coupon_discount),SUM(trans_item_merge.total_discount),SUM(trans_item_merge.original_price),SUM(trans_item_merge.MRP),SUM(trans_item_merge.correctSellingPrice),SUM(trans_item_merge.coupon_dis_per),SUM(trans_item_merge.other_dis_per),SUM(trans_item_merge.total_dis_per),SUM(trans_item_merge.MRP>category),SUM(trans_item_merge.typebrand/customer),SUM(trans_item_merge.day),STD(trans_item_merge.quantity),STD(trans_item_merge.selling_price),STD(trans_item_merge.other_discount),STD(trans_item_merge.coupon_discount),STD(trans_item_merge.total_discount),STD(trans_item_merge.original_price),STD(trans_item_merge.MRP),STD(trans_item_merge.correctSellingPrice),STD(trans_item_merge.coupon_dis_per),STD(trans_item_merge.other_dis_per),STD(trans_item_merge.total_dis_per),STD(trans_item_merge.MRP>category),STD(trans_item_merge.typebrand/customer),STD(trans_item_merge.day),MAX(trans_item_merge.quantity),MAX(trans_item_merge.selling_price),MAX(trans_item_merge.other_discount),MAX(trans_item_merge.coupon_discount),MAX(trans_item_merge.total_discount),MAX(trans_item_merge.original_price),MAX(trans_item_merge.MRP),MAX(trans_item_merge.correctSellingPrice),MAX(trans_item_merge.coupon_dis_per),MAX(trans_item_merge.other_dis_per),MAX(trans_item_merge.total_dis_per),...,MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.day)),MEAN(trans_item_merge.item.MIN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MIN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MIN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MIN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MIN(trans_item_merge.day)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.day)),MEAN(trans_item_merge.item.COUNT(trans_item_merge)),MEAN(trans_item_merge.item.NUM_UNIQUE(trans_item_merge.customer_id)),NUM_UNIQUE(trans_item_merge.item.MODE(trans_item_merge.customer_id)),MODE(trans_item_merge.item.MODE(trans_item_merge.customer_id))
0,1,1227,98513.93,17030.40,2116.83,19147.23,1176

In [15]:
# feature_matrix_train.head()
# feature_matrix_train.id.dtype
feature_matrix_train.id.dtype, master_data.id.dtype
# feature_matrix_train.shape


(dtype('int64'), dtype('int64'))

In [16]:
master_data.customer_id.dtype, feature_matrix_customer.customer_id.dtype

(dtype('int64'), dtype('int64'))

In [17]:
# merging feature tools feature back to master_data 
# master_data['id'] = master_data['id'].astype(str)
feature_matrix_train_sub = feature_matrix_train.drop(['campaign_id', 'coupon_id', 'customer_id', 'campaign_type'], axis=1)
master_data = pd.merge(master_data, feature_matrix_train_sub, on='id', how='left') # merging training features

# master_data['coupon_id'] = master_data['coupon_id'].astype(str)
# master_data = pd.merge(master_data, feature_matrix_1, on='coupon_id', how='left') # merging coupon features

master_data = pd.merge(master_data, feature_matrix_customer, on='customer_id', how='left') # merging customer transaction features
master_data.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,SUM(coupons.brand_type),STD(coupons.brand_type),MAX(coupons.brand_type),SKEW(coupons.brand_type),MIN(coupons.brand_type),MEAN(coupons.brand_type),COUNT(coupons),NUM_UNIQUE(coupons.item_id),NUM_UNIQUE(coupons.brand),NUM_UNIQUE(coupons.category),MODE(coupons.item_id),MODE(coupons.brand),MODE(coupons.category),age_range,marital_status,rented,family_size,no_of_children,income_bracket,DAY(start_date),DAY(end_date),YEAR(start_date),YEAR(end_date),MONTH(start_date),MONTH(end_date),WEEKDAY(start_date),WEEKDAY(end_date),campaign.SUM(trainingdata.duration),campaign.SUM(trainingdata.SUM(coupons.brand_type)),campaign.SUM(trainingdata.STD(coupons.brand_type)),campaign.SUM(trainingdata.MAX(coupons.brand_type)),...,MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.day)),MEAN(trans_item_merge.item.MIN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MIN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MIN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MIN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MIN(trans_item_merge.day)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.day)),MEAN(trans_item_merge.item.COUNT(trans_item_merge)),MEAN(trans_item_merge.item.NUM_UNIQUE(trans_item_merge.customer_id)),NUM_UNIQUE(trans_item_merge.item.MODE(trans_item_merge.customer_id)),MODE(trans_item_merge.item.MODE(trans_item_merge.customer_id))
0,1,13,27,1053,0.0,X,2013-05-19,2013-07-05,-47,125,0.0,1,0.0,1,1.0,125,125,2,1,10535,1105,Grocery,1.0,NaN,0.0,1.0,NaN,5.0,19,5,2013,2013,5,7,6,4,-1062482,3164965,293.594784,16197,...,-1.266933,2.842173,2.317235,0.823386,0.953052,-0.001050,0.0,0.035996,1.745161,120.486871,5.093645,0.000000,5.093645,151.421452,149.665416,114.076387,0.000000,0.975401,0.995741,0.270968,0.535484,1.706452,527.286000,206.505559,25.193313,0.453469,25.646782,232.152341,163.102791,206.052090,0.174482,12.387128,12.561610,0.305008,0.535484,15.672901,916.909677,160.332258,160,1405
1,2,13,116,48,0.0,X,2013-05-19,2013-07-05,-47,0,0.0,0,0.0,0,0.0,3,3,1,1,36721,56,

In [20]:
master_data.shape

(128595, 1059)

# **Model building** 

### missing value treatment

Note: missing columns

age_range,
marital_status,
rented,
family_size,
no_of_children,
income_bracket,
customer_id_redeem_index,
coupon_dis_per_avg 

filling marital status

In [0]:
# master_data['marital_status'] = master_data.apply(lambda x: 'Single' if ((x.family_size =='1') & (str(x.marital_status)=="nan")) else x.marital_status, axis=1)
# # master_data[(master_data.family_size=='1')&(master_data.marital_status.isnull())].shape

In [0]:
# master_data.loc[(master_data['family_size'] != '1') & (master_data['marital_status'].isnull()), 'marital_status'] = 'Married' 
# # master_data.marital_status.isnull().sum()

In [0]:
# # filling no of children
# master_data.loc[(master_data['family_size'] == '1') & (master_data['no_of_children'].isnull()), 'no_of_children'] = '0' 
# # master_data[(master_data.family_size=='1')&(master_data.no_of_children.isnull())].shape


**model**

Some observations:
1. Startified Kfold > Grouped K fold > K fold> no fold for cv
2. Label encoding give you a LB boost of 0.3-0.5
3. Ensemble gives you a LB boost of 0.5-1


In [21]:
# master_data.columns.tolist(), 
master_data.shape

(128595, 1059)

In [0]:
pd.options.display.max_rows = 999
master_data.dtypes

In [22]:
master_data.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'campaign_type', 'start_date', 'end_date', 'duration',
       'SUM(coupons.brand_type)',
       ...
       'MEAN(trans_item_merge.item.NUM_UNIQUE(trans_item_merge.customer_id))',
       'NUM_UNIQUE(trans_item_merge.item.MODE(trans_item_merge.customer_id))',
       'MODE(trans_item_merge.item.MODE(trans_item_merge.customer_id))',
       'count_coupon_id', 'campaign_days', 'campaign_type_redeem_index',
       'customer_id_redeem_index', 'unique_coupon_id', 'total_coupon_redeem',
       'redeem_per_coupon'],
      dtype='object', length=1059)

In [0]:
# all_cols = ['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status','age_range', 'marital_status', 'rented', 'family_size','no_of_children', \
#             'income_bracket', 'campaign_type', 'start_date','end_date', 'rented_mean', 'income_bracket_sum', 'age_range_mean','family_size_mean', 'no_of_children_mean',\
#             'no_of_children_count','marital_status_count', 'count_coupon_id', 'campaign_days','campaign_type_redeem_index', 'customer_id_redeem_index',\
#             'unique_coupon_id', 'total_coupon_redeem', 'redeem_per_coupon','unique_items', 'unique_brands', 'unique_category', 'unique_brand_type','mostoccuredbrand', \
#             'mostoccuredcategory', 'mostoccuredbrand_type','Bakery', 'Dairy, Juices & Snacks', 'Flowers & Plants', 'Garden','Grocery', 'Meat', 'Miscellaneous', \
#             'Natural Products', 'Packaged Meat','Pharmaceutical', 'Prepared Food', 'Restauarant', 'Salads', 'Seafood','Skin & Hair Care', 'Travel', 'Vegetables (cut)', \
#             'typebrand_mean','category_mean', 'category_count', 'category_max', 'category_min','brand_mean', 'brand_count', 'brand_min', 'brand_max', 'no_transaction',\
#             'total_quantity', 'total_selling_price', 'total_other_discount','total_coupon_discount', 'total_total_discount', 'unique_items_cus','unique_brand_cus', \
#             'unique_category_cus', 'coupon_dis_per_avg','other_dis_per_avg', 'total_dis_per_avg', 'quantity_mean','coupon_discount_mean', 'other_discount_mean', \
#             'day_mean','selling_price_mean', 'selling_price_min', 'selling_price_max','selling_price_unique', 'MRP_mean', 'MRP_sum', 'MRP_min', 'MRP_max','MRP_unique', \
#             'correctSellingPrice_mean', 'correctSellingPrice_sum','correctSellingPrice_min', 'correctSellingPrice_max','correctSellingPrice_unique', 'MRP_cat_mean',\
#             'typebrand_per_customer_mean', 'mostpurchaseditem','mostpurchasedbrand', 'mostpurchasedcategory','total_discounted_trans']

drop_cols = ['id','coupon_id','start_date','end_date','campaign_type','customer_id_redeem_index','total_coupon_redeem','redeem_per_coupon']


In [24]:
final_model_data = master_data.copy()
final_model_data.drop(drop_cols, axis=1, inplace=True)
final_model_data.shape


(128595, 1051)

In [0]:
# converting some columns to categorical variables
# customer_id
# rented
# income_bracket
# mostpurchaseditem
# mostpurchasedbrand

final_model_data['customer_id'] = final_model_data['customer_id'].astype(str)
final_model_data['rented'] = final_model_data['rented'].astype(str)
final_model_data['income_bracket'] = final_model_data['income_bracket'].astype(str)
# final_model_data['mostpurchasedbrand'] = final_model_data['mostpurchasedbrand'].astype(str)

In [0]:
# filling missing rate columns
final_model_data.age_range.fillna('-999', inplace=True)
final_model_data.marital_status.fillna('-999', inplace=True)
final_model_data.rented.fillna('-999', inplace=True)
final_model_data.family_size.fillna('-999', inplace=True)
final_model_data.no_of_children.fillna('-999', inplace=True)
final_model_data.income_bracket.fillna('-999', inplace=True)

# final_model_data.rented_mean.fillna(-999, inplace=True)
# final_model_data.income_bracket_sum.fillna(-999, inplace=True)
# final_model_data.age_range_mean.fillna(-999, inplace=True)
# final_model_data.family_size_mean.fillna(-999, inplace=True)

# final_model_data.no_of_children_mean.fillna(-999, inplace=True)
# final_model_data.no_of_children_count.fillna(-999, inplace=True)
# final_model_data.marital_status_count.fillna(-999, inplace=True)
                                    
final_model_data.count_coupon_id.fillna(-999, inplace=True)                                     
final_model_data.unique_coupon_id.fillna(-999, inplace=True)
# final_model_data.coupon_dis_per_avg.fillna(-999, inplace=True)

In [0]:
final_model_data.isnull().sum()

In [0]:
final_model_data.fillna(-999, inplace=True)

In [0]:
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder

# #One Hot Encoding of the Categorical features 
# one_hot_age_range=pd.get_dummies(final_model_data.age_range, prefix='age_range') 
# one_hot_marital_status=pd.get_dummies(final_model_data.marital_status, prefix='marital_status') 
# one_hot_family_sizes=pd.get_dummies(final_model_data.family_size,prefix='family_size') 
# one_hot_no_of_children=pd.get_dummies(final_model_data.no_of_children,prefix='no_of_children')
# one_hot_income_bracket=pd.get_dummies(final_model_data.income_bracket,prefix='income_bracket') 
# one_hot_rented=pd.get_dummies(final_model_data.rented,prefix='rented') 
# one_hot_campaign_type=pd.get_dummies(final_model_data.campaign_type,prefix='campaign_type') 

# final_model_data.drop(['age_range', 'marital_status','rented' ,'family_size','no_of_children','income_bracket', 'campaign_type'], axis=1,inplace=True)

#Merging one hot encoded features with our dataset 'data' 
# final_model_data=pd.concat([final_model_data,one_hot_age_range,one_hot_marital_status,one_hot_family_sizes,one_hot_no_of_children,\
#                 one_hot_income_bracket,one_hot_rented, one_hot_campaign_type],axis=1) 

In [99]:
final_model_data.head()

,campaign_id,customer_id,redemption_status,count_coupon_id,campaign_days,campaign_type_redeem_index,unique_coupon_id,duration,SUM(coupons.brand_type),STD(coupons.brand_type),MAX(coupons.brand_type),SKEW(coupons.brand_type),MIN(coupons.brand_type),MEAN(coupons.brand_type),COUNT(coupons),NUM_UNIQUE(coupons.item_id),NUM_UNIQUE(coupons.brand),NUM_UNIQUE(coupons.category),MODE(coupons.item_id),MODE(coupons.brand),MODE(coupons.category),age_range,marital_status,rented,family_size,no_of_children,income_bracket,DAY(start_date),DAY(end_date),YEAR(start_date),YEAR(end_date),MONTH(start_date),MONTH(end_date),WEEKDAY(start_date),WEEKDAY(end_date),campaign.SUM(trainingdata.duration),campaign.SUM(trainingdata.SUM(coupons.brand_type)),campaign.SUM(trainingdata.STD(coupons.brand_type)),campaign.SUM(trainingdata.MAX(coupons.brand_type)),campaign.SUM(trainingdata.SKEW(coupons.brand_type)),...,MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.day)),MEAN(trans_item_merge.item.MIN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MIN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MIN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MIN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MIN(trans_item_merge.day)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.day)),MEAN(trans_item_merge.item.COUNT(trans_item_merge)),MEAN(trans_item_merge.item.NUM_UNIQUE(trans_item_merge.customer_id)),NUM_UNIQUE(trans_item_merge.item.MODE(trans_item_merge.customer_id)),MODE(trans_item_merge.item.MODE(trans_item_merge.customer_id))
0,13,1053,0.0,122.0,47,0.872428,30.0,-47,125,0.0,1,0.0,1,1.0,125,125,2,1,10535,1105,Grocery,1,-999,0.0,1,-999,5.0,19,5,2013,2013,5,7,6,4,-1062482,3164965,293.594784,16197,-1228.607336,...,-1.266933,2.842173,2.317235,0.823386,0.953052,-0.001050,0.0,0.035996,1.745161,120.486871,5.093645,0.000000,5.093645,151.421452,149.665416,114.076387,0.000000,0.975401,0.995741,0.270968,0.535484,1.706452,527.286000,206.505559,25.193313,0.453469,25.646782,232.152341,163.102791,206.052090,0.174482,12.387128,12.561610,0.305008,0.535484,15.672901,916.909677,160.332258,160,14

# **CatBoost Model**

In [100]:
col_ids = ['redemption_status', 'campaign_id']
# cols_drop_exp = ['Grocery', 'Bakery', 'Skin & Hair Care', 'Pharmaceutical','Seafood', 'Packaged Meat', 'Dairy, Juices & Snacks','Natural Products', 'Miscellaneous', \
#                  'Prepared Food', 'Meat','Vegetables (cut)', 'Travel', 'Garden', 'Flowers & Plants','Salads', 'Restauarant']

##Creating validation set
X_train = final_model_data[(final_model_data.campaign_id.isin([1,2,3,4,5,6,7,8,9,10,12,13,27, 28]))]
X_val = final_model_data[(final_model_data.campaign_id.isin([26,29,11,30]))]
X_full = final_model_data[~(final_model_data.redemption_status.isnull())]
X_test = final_model_data[(final_model_data.redemption_status.isnull())]

y_train = X_train["redemption_status"]
y_val = X_val["redemption_status"]
y_full = X_full["redemption_status"]
X_train.drop(col_ids,axis=1,inplace=True)
X_val.drop(col_ids,axis=1,inplace=True)
X_full.drop(col_ids,axis=1,inplace=True)
X_test.drop(col_ids,axis=1,inplace=True)

X_train.shape, X_val.shape, X_full.shape, X_test.shape

((59169, 1049), (19200, 1049), (128595, 1049), (0, 1049))

In [101]:
X_train.head()

,customer_id,count_coupon_id,campaign_days,campaign_type_redeem_index,unique_coupon_id,duration,SUM(coupons.brand_type),STD(coupons.brand_type),MAX(coupons.brand_type),SKEW(coupons.brand_type),MIN(coupons.brand_type),MEAN(coupons.brand_type),COUNT(coupons),NUM_UNIQUE(coupons.item_id),NUM_UNIQUE(coupons.brand),NUM_UNIQUE(coupons.category),MODE(coupons.item_id),MODE(coupons.brand),MODE(coupons.category),age_range,marital_status,rented,family_size,no_of_children,income_bracket,DAY(start_date),DAY(end_date),YEAR(start_date),YEAR(end_date),MONTH(start_date),MONTH(end_date),WEEKDAY(start_date),WEEKDAY(end_date),campaign.SUM(trainingdata.duration),campaign.SUM(trainingdata.SUM(coupons.brand_type)),campaign.SUM(trainingdata.STD(coupons.brand_type)),campaign.SUM(trainingdata.MAX(coupons.brand_type)),campaign.SUM(trainingdata.SKEW(coupons.brand_type)),campaign.SUM(trainingdata.MIN(coupons.brand_type)),campaign.SUM(trainingdata.MEAN(coupons.brand_type)),...,MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.SKEW(trans_item_merge.day)),MEAN(trans_item_merge.item.MIN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MIN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MIN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MIN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MIN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MIN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MIN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MIN(trans_item_merge.day)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.quantity)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.selling_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_discount)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.original_price)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.correctSellingPrice)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.coupon_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.other_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.total_dis_per)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.MRP>category)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.typebrand/customer)),MEAN(trans_item_merge.item.MEAN(trans_item_merge.day)),MEAN(trans_item_merge.item.COUNT(trans_item_merge)),MEAN(trans_item_merge.item.NUM_UNIQUE(trans_item_merge.customer_id)),NUM_UNIQUE(trans_item_merge.item.MODE(trans_item_merge.customer_id)),MODE(trans_item_merge.item.MODE(trans_item_merge.customer_id))
0,1053,122.0,47,0.872428,30.0,-47,125,0.0,1,0.0,1,1.0,125,125,2,1,10535,1105,Grocery,1,-999,0.0,1,-999,5.0,19,5,2013,2013,5,7,6,4,-1062482,3164965,293.594784,16197,-1228.607336,15489,16030.829404,...,-1.266933,2.842173,2.317235,0.823386,0.953052,-0.001050,0.0,0.035996,1.745161,120.486871,5.093645,0.000000,5.093645,151.421452,149.665416,114.076387,0.000000,0.975401,0.995741,0.270968,0.535484,1.706452,527.286000,206.505559,25.193313,0.453469,25.646782,232.152341,163.102791,206.052090,

In [103]:
#Catboost
categorical_features_indices = np.where((X_train.dtypes != 'int16') & (X_train.dtypes != 'int8') & (X_train.dtypes != 'int64') & (X_train.dtypes != 'float16') & (X_train.dtypes != 'float64'))[0]
m=CatBoostClassifier(iterations= 2000,loss_function='CrossEntropy',
                             od_type='Iter', 
                             od_wait=1000, eval_metric = 'AUC', bagging_temperature=0.7232969440547752, depth=6.0, l2_leaf_reg=8.51645749792985,
                             random_strength=20,random_seed=100,task_type = 'GPU',metric_period = 40)
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)], early_stopping_rounds=200,verbose=40,cat_features=categorical_features_indices)
print(m.get_feature_importance(prettified=True))


# m=CatBoostClassifier(iterations= 2000,loss_function='CrossEntropy',
#                              od_type='Iter', od_wait=300, eval_metric = 'AUC', bagging_temperature=0, depth=4.0, l2_leaf_reg=8.51645749792985,
#                              random_strength=20,random_seed=100,task_type = 'GPU',metric_period = 40)

0:	learn: 0.6188787	test: 0.5530058	best: 0.5530058 (0)	total: 133ms	remaining: 4m 25s
40:	learn: 0.9169331	test: 0.7523719	best: 0.7549567 (37)	total: 3.82s	remaining: 3m 2s
80:	learn: 0.9325014	test: 0.8112511	best: 0.8145750 (77)	total: 7.4s	remaining: 2m 55s
120:	learn: 0.9390024	test: 0.8307118	best: 0.8338697 (114)	total: 10.9s	remaining: 2m 49s
160:	learn: 0.9439876	test: 0.8364100	best: 0.8364907 (159)	total: 14.5s	remaining: 2m 45s
200:	learn: 0.9473209	test: 0.8361471	best: 0.8364907 (159)	total: 18s	remaining: 2m 41s
240:	learn: 0.9503846	test: 0.8456861	best: 0.8456861 (240)	total: 21.5s	remaining: 2m 36s
280:	learn: 0.9529657	test: 0.8513393	best: 0.8522624 (277)	total: 25s	remaining: 2m 32s
320:	learn: 0.9552269	test: 0.8552847	best: 0.8552847 (320)	total: 28.6s	remaining: 2m 29s
360:	learn: 0.9572638	test: 0.8546184	best: 0.8564076 (328)	total: 32.1s	remaining: 2m 25s
400:	learn: 0.9592285	test: 0.8526561	best: 0.8564076 (328)	total: 35.7s	remaining: 2m 22s
440:	learn: 0

In [0]:
#Catboost
categorical_features_indices = np.where((X_full.dtypes != 'int16') & (X_full.dtypes != 'int8') & (X_full.dtypes != 'int64') & (X_full.dtypes != 'float16') & (X_full.dtypes != 'float64'))[0]
m=CatBoostClassifier(iterations= 1500,loss_function='CrossEntropy', eval_metric = 'AUC', bagging_temperature=0.7232969440547752, depth=6.0, l2_leaf_reg=8.51645749792985,
                             random_strength=20,random_seed=100,task_type = 'GPU',metric_period = 40)
m.fit(X_full,y_full,verbose=40,cat_features=categorical_features_indices)
print(m.get_feature_importance(prettified=True))



In [0]:
y_pred = m.predict_proba(X_test)[:,1]
submission = pd.DataFrame({'id':test_data['id'], 'redemption_status':y_pred})


In [0]:
# submission.sort_values('redemption_status', ascending=False)
submission[submission.redemption_status>0.18].shape, submission[submission.redemption_status<0.0001].shape



((227, 2), (6084, 2))

In [0]:
submission.to_csv('catboost_single_v2.csv', index=False)
from google.colab import files
files.download('catboost_single_v2.csv')


**Stratified Catboost**

In [0]:
X_train.columns

In [27]:
##Creating validation set
X_train = final_model_data[(final_model_data.campaign_id.isin([1,2,3,4,5,6,7,8,9,10,12,13,27, 28]))]
X_val = final_model_data[(final_model_data.campaign_id.isin([26,29,11,30]))]
X_full = final_model_data[~(final_model_data.redemption_status.isnull())]
X_test = final_model_data[(final_model_data.redemption_status.isnull())]

X_full.shape, X_test.shape


((78369, 1051), (50226, 1051))

In [0]:
train_cols = X_train.columns.tolist()
train_cols.remove('campaign_id')
train_cols.remove('redemption_status')


In [109]:
len(train_cols)


1049

In [29]:
# Model
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from catboost import Pool, CatBoostClassifier
from category_encoders import TargetEncoder
from scipy.special import logit
#from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model', n_folds=5):
    kf = StratifiedKFold(n_splits=n_folds, shuffle = True, random_state = 228)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    feature_importances = pd.DataFrame()
    feature_importances['feature'] = test.columns
    i = 1
    for dev_index, val_index in fold_splits:
        print('-------------------------------------------')
        print('Started ' + label + ' fold ' + str(i) + f'/{n_folds}')
        dev_X, val_X = train.iloc[dev_index], train.iloc[val_index]
        dev_y, val_y = target.iloc[dev_index], target.iloc[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y, fi = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        feature_importances[f'fold_{i}'] = fi
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score), '\n')
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std score : {}'.format(label, np.std(cv_scores)))
    pred_full_test = pred_full_test / n_folds
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores, 'fi': feature_importances}
    return results


def runCAT(train_X, train_y, test_X, test_y, test_X2, params):
    # Pool the data and specify the categorical feature indices
    print('Pool Data')
    _train = Pool(train_X, train_y,categorical_features_indices)
    _valid = Pool(test_X, test_y,categorical_features_indices)    
    print('Train CAT')
    model = CatBoostClassifier(**params)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=1000,
                          plot=False)
    feature_im = fit_model.feature_importances_
    print('Predict 1/2')
    pred_test_y = logit(fit_model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(fit_model.predict_proba(test_X2)[:, 1])
    return pred_test_y, pred_test_y2, feature_im

# Use some baseline parameters
cat_params = {'loss_function': 'CrossEntropy', 
              'eval_metric': "AUC",
              'iterations': 2000,'bagging_temperature':0.7232969440547752, 'depth':6.0, 
              'l2_leaf_reg':8.51645749792985,'random_strength':20,'random_seed':100,
              'task_type':'GPU','metric_period':40,
              'od_type': "Iter",
              'early_stopping_rounds': 150,
             }
categorical_features_indices = np.where((X_full[train_cols].dtypes != 'int16') & (X_full[train_cols].dtypes != 'int8') & (X_full[train_cols].dtypes != 'int64') & (X_full[train_cols].dtypes != 'float16') & (X_full[train_cols].dtypes != 'float64'))[0]

n_folds = 10
results = run_cv_model(X_full[train_cols].fillna(-999), X_test[train_cols].fillna(-999), X_full['redemption_status'], runCAT, cat_params, auc, 'cat', n_folds=n_folds)
tmp = dict(zip(test_data.id.values, results['test']))      

-------------------------------------------
Started cat fold 1/10
Pool Data
Train CAT
0:	learn: 0.8048752	test: 0.7393026	best: 0.7393026 (0)	total: 91.7ms	remaining: 3m 3s
1000:	learn: 0.9733759	test: 0.9303724	best: 0.9303970 (991)	total: 1m 4s	remaining: 1m 4s
bestTest = 0.931224525
bestIteration = 1094
Shrink model to first 1095 iterations.
Predict 1/2
Predict 2/2
cat cv score 1: 0.9312245488485669 

-------------------------------------------
Started cat fold 2/10
Pool Data
Train CAT
0:	learn: 0.7951960	test: 0.8347192	best: 0.8347192 (0)	total: 88ms	remaining: 2m 55s
1000:	learn: 0.9710495	test: 0.9521853	best: 0.9521853 (1000)	total: 1m 5s	remaining: 1m 5s
1999:	learn: 0.9870111	test: 0.9587982	best: 0.9587982 (1999)	total: 2m 9s	remaining: 0us
bestTest = 0.9587982297
bestIteration = 1999
Shrink model to first 2000 iterations.
Predict 1/2
Predict 2/2
cat cv score 2: 0.9587982469141031 

-------------------------------------------
Started cat fold 3/10
Pool Data
Train CAT
0:	lear

In [0]:
answer1 = pd.DataFrame()
answer1['id'] = test_data.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv('stratified_catboost_featureTools_v2.csv', index = None)

In [0]:
from google.colab import files
files.download('stratified_catboost_featureTools_v2.csv')

**LightGbm**

In [0]:
col_ids = ['redemption_status', 'campaign_id']

##Creating validation set
X_train = final_model_data[(final_model_data.campaign_id.isin([1,2,3,4,5,6,7,8,9,10,12,13,27, 28]))]
X_val = final_model_data[(final_model_data.campaign_id.isin([26,29,11,30]))]
X_full = final_model_data[~(final_model_data.redemption_status.isnull())]
X_test = final_model_data[(final_model_data.redemption_status.isnull())]

y_train = X_train["redemption_status"]
y_val = X_val["redemption_status"]
y_full = X_full["redemption_status"]
X_train.drop(col_ids,axis=1,inplace=True)
X_val.drop(col_ids,axis=1,inplace=True)
X_full.drop(col_ids,axis=1,inplace=True)
X_test.drop(col_ids,axis=1,inplace=True)

X_train.shape, X_val.shape, X_full.shape, X_test.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


((59169, 87), (19200, 87), (78369, 87), (50226, 87))

In [0]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
#With Catgeorical Features
cate_features_name = ['customer_id','age_range','marital_status','rented','family_size','no_of_children','income_bracket','mostoccuredcategory',\
                      'mostoccuredbrand_type','mostpurchasedbrand','mostpurchasedcategory','mostpurchasedcategory']

for i in cate_features_name:
  l = LabelEncoder()
  X_train[i] = l.fit_transform(X_train[i].astype(str))
  X_val[i] = l.fit_transform(X_val[i].astype(str))
  X_test[i] = l.fit_transform(X_test[i].astype(str))
  X_full[i] = l.fit_transform(X_full[i].astype(str))
  

In [0]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
                        n_estimators = 20, num_leaves = 38)


param_grid = {
    'n_estimators': [x for x in range(24,40,2)],
    'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
        eval_set = [(X_val, y_val)],
        eval_metric = ['auc', 'binary_logloss'],
        early_stopping_rounds = 50)

In [0]:
gbm = lgb.LGBMClassifier(learning_rate = 0.1, metric = 'l1', 
                        n_estimators = 24)


gbm.fit(X_full, y_full,
        eval_set=[(X_val, y_val)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=100, verbose=100)


Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[24]	valid_0's auc: 0.982907	valid_0's l1: 0.0105565	valid_0's binary_logloss: 0.0199978


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               metric='l1', min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=24, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [0]:
from scipy.special import logit, expit
y_pred_prob_test = gbm.predict_proba(X_test)[:, 1]
submission = pd.DataFrame({'id':test_data['id'], 'redemption_status_lgb':y_pred_prob_test})
submission['redemption_status_lgb'] = logit(submission['redemption_status_lgb'])
submission.head()


,id,redemption_status_lgb
0,3,-1.657556
1,4,-5.439989
2,5,-3.250819
3,8,-6.822548
4,10,-6.822548


In [0]:
answer1.rename(columns={'redemption_status':'redemption_status_cat'}, inplace=True)
answer1.head()


,id,redemption_status_cat
0,3,-2.309871
1,4,-6.504001
2,5,-2.283393
3,8,-8.095214
4,10,-8.009622


In [0]:
# Ensemble of Lightgbm single and catboost stratified model
Ensemble = pd.merge(answer1, submission, on='id')
Ensemble['redemption_status'] = (Ensemble['redemption_status_lgb']+Ensemble['redemption_status_cat'])/2
Ensemble = Ensemble[['id', 'redemption_status']]
Ensemble.head()

,id,redemption_status
0,3,-1.983714
1,4,-5.971995
2,5,-2.767106
3,8,-7.458881
4,10,-7.416085


In [0]:
Ensemble.to_csv('Ensemble_stratifiedCat_SingleLGB.csv', index=False)
from google.colab import files
files.download('Ensemble_stratifiedCat_SingleLGB.csv')